# Exploring Cython

In this notebook we will explore accelerating computations using Cython.

### Monte-Carlo simulation of dice rolls

We will look to use monte-carlo simulation to compute the probability of seeing *at least* M sixes from N dices rolls. 

Below is a basic Python implementation of such a simulation.

In [1]:
from datetime import datetime
import random

def is_six(ndice, nsix):
    six = 0
    for j in range(ndice):
        r = random.randint(1, 6)  # roll jth die
        if r == 6:
            six += 1

    if six >= nsix:
        return 1
    return 0
    
def dice6_py(N, ndice, nsix):
    M = 0                     # no of successful events
    for i in range(N):        # repeat N experiments
        M += is_six(ndice, nsix)
    p = float(M)/N
    return p

In [8]:
from scipy.stats import binom

# Computes the true probability of rolling M sixes in N dice throws
def true_prob(ndice, nsix):
    p = 0.0
    for n in range(nsix, ndice+1):
        p += binom.pmf(n, ndice, 1/6)
    return p

We can now run the computation and compare it to the true probability of the event.

In [ ]:
N = 500000
ndice = 20
nsix = 5
t0 = datetime.now()
p = dice6_py(N, ndice, nsix)
t1 = datetime.now()
print(f"Time     : {t1-t0}\nMC prob  : {p:.6f}\nTrue prob: {true_prob(ndice, nsix):.6f}")

Pretty slow. We now see how we can speed it up.

### Vectorized implementation

The first thing we can try to accelerate the code is to vectorize it using numpy. Below is a vectorized implementation.

In [ ]:
import numpy as np

def dice6_vec2(N, ndice, nsix):
    eyes = np.random.randint(1, 6+1, (N, ndice))
    six = [6 for i in range(ndice)]
    M = 0
    for i in range(N):
        compare = eyes[i,:] == six
        if np.sum(compare) >= nsix:
            M += 1
    p = float(M)/N
    return p

In [ ]:
N = 500000
ndice = 20
nsix = 5
t0 = datetime.now()
p = dice6_vec2(N, ndice, nsix)
t1 = datetime.now()
print(f"Time     : {t1-t0}\nMC prob  : {p:.6f}\nTrue prob: {true_prob(ndice, nsix):.6f}")

### Plain Cython Implementation

We now turn to doing the same computation in Cython. 

TODO: 

Write a naive implementation of our dice computation in a function `dice6_cy1()` in the Cython file `dice6_cy1.pyx`. To compute a random integer, use the Python `random` library in your Cython code and also write the loop using `range()`.

Once that is done, write a setup script in `dice6_cy1_setup.py` to allow compilation.


In [2]:
from datetime import datetime

# Compile and import Cython code
!python dice6_cy1_setup.py build_ext --inplace
from dice6_cy1 import dice6_cy1

running build_ext
cythoning dice6_cy1.pyx to dice6_cy1.c
/home/users/u6642247/anaconda3/envs/ML/lib/python3.10/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/users/u6642247/Python-HPC/part2/solutions/dice6_cy1.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'dice6_cy1' extension
creating build
creating build/temp.linux-x86_64-cpython-310
gcc -pthread -B /home/users/u6642247/anaconda3/envs/ML/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/users/u6642247/anaconda3/envs/ML/include -fPIC -O2 -isystem /home/users/u6642247/anaconda3/envs/ML/include -fPIC -I/home/users/u6642247/anaconda3/envs/ML/include/python3.10 -c dice6_cy1.c -o build/temp.linux-x86_64-cpython-310/dice6_cy1.o
gcc -pthread -B /home/users/u6642247/anaconda3/envs/ML/compiler_compat -shared -Wl,-rpath,/home/users/u6642247/anaconda3/

In [4]:
# Run
N = 500000
ndice = 20
nsix = 5
t0 = datetime.now()
p = dice6_cy1(N, ndice, nsix)
t1 = datetime.now()
print(f"Time     : {t1-t0}\nMC prob  : {p:.6f}")

Time     : 0:00:05.206797
MC prob  : 0.231544


While it is often faster than the naive Python implementation, it is still slower than our vectorized Python implementation.

We can use a profiler to see which parts of the cython code take the most time.

In [5]:
import cProfile, pstats

statement = "dice6_cy1(N, ndice, nsix)"

cProfile.runctx(statement, globals(), locals(), '.prof')
s = pstats.Stats('.prof')
s.strip_dirs().sort_stats('time').print_stats(30)

Fri Mar 24 16:17:41 2023    .prof

         83833818 function calls in 19.154 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 10000000    7.352    0.000   15.302    0.000 random.py:292(randrange)
 10000000    4.643    0.000    6.358    0.000 random.py:239(_randbelow_with_getrandbits)
 10000000    2.373    0.000   17.675    0.000 random.py:366(randint)
 30000000    1.592    0.000    1.592    0.000 {built-in method _operator.index}
   500000    1.391    0.000   19.066    0.000 dice6_cy1.pyx:5(is_six)
 13333813    1.044    0.000    1.044    0.000 {method 'getrandbits' of '_random.Random' objects}
 10000000    0.671    0.000    0.671    0.000 {method 'bit_length' of 'int' objects}
        1    0.088    0.088   19.154   19.154 dice6_cy1.pyx:16(dice6_cy1)
        1    0.000    0.000   19.154   19.154 {built-in method builtins.exec}
        1    0.000    0.000   19.154   19.154 <string>:1(<module>)
        1    0.000    0.000   1

Looking at the cumtime (cumulative time) column, we can see that the function spends most of its time making calls out to the Python `random.py` library. We can thus try speed things up by using the C standard library random number generator.

TODO:

Try implement our dice roll simulation in `dice6_cy2.pyx` using the C standard library function `rand()` and its setup script in `dice6_cy2_setup.py`.

 Note that `rand()` returns an integer between 0 and `RAND_MAX` so you will need to try and restrict its output to 1 to 6 only. 

In [6]:
!python dice6_cy2_setup.py build_ext --inplace
from dice6_cy2 import dice6_cy2

running build_ext
cythoning dice6_cy2.pyx to dice6_cy2.c
/home/users/u6642247/anaconda3/envs/ML/lib/python3.10/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/users/u6642247/Python-HPC/part2/solutions/dice6_cy2.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'dice6_cy2' extension
gcc -pthread -B /home/users/u6642247/anaconda3/envs/ML/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/users/u6642247/anaconda3/envs/ML/include -fPIC -O2 -isystem /home/users/u6642247/anaconda3/envs/ML/include -fPIC -I/home/users/u6642247/anaconda3/envs/ML/include/python3.10 -c dice6_cy2.c -o build/temp.linux-x86_64-cpython-310/dice6_cy2.o
gcc -pthread -B /home/users/u6642247/anaconda3/envs/ML/compiler_compat -shared -Wl,-rpath,/home/users/u6642247/anaconda3/envs/ML/lib -Wl,-rpath-link,/home/users/u6642247/anaconda3/e

In [9]:
from datetime import datetime


N = 500000
ndice = 100
nsix = 20
t0 = datetime.now()
p = dice6_cy2(N, ndice, nsix)
t1 = datetime.now()
print(f"Time     : {t1-t0}\nMC prob  : {p:.6f}\nTrue prob: {true_prob(ndice, nsix):.6f}")

Time     : 0:00:01.333889
MC prob  : 0.217732
True prob: 0.219750


Much faster! But perhaps we can do better...

## Parallelization in Cython

We now try doing accelerate things using parallelisation in Cython. In theory this will allow us to run multiple iterations of the loop at once in different threads.

TODO:

Implement our dice simulation in `dice6_cy3.pyx` and `dice6_cy3_setup.py` using the Cython parallel library to parallelise our MC simulations.

NOTE: Typecasting of C variables in Cython is done by placing the type in angle brackets, e.g. if x is a double, <int> x typecasts x into an integer.

NOTE: Make sure to link the openmp library in `dice6_cy3_setup.py`.

In [10]:
!python dice6_cy3_setup.py build_ext --inplace
from dice6_cy3 import dice6_cy3

running build_ext
cythoning dice6_cy3.pyx to dice6_cy3.c
/home/users/u6642247/anaconda3/envs/ML/lib/python3.10/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/users/u6642247/Python-HPC/part2/solutions/dice6_cy3.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'dice6_cy3' extension
gcc -pthread -B /home/users/u6642247/anaconda3/envs/ML/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/users/u6642247/anaconda3/envs/ML/include -fPIC -O2 -isystem /home/users/u6642247/anaconda3/envs/ML/include -fPIC -I/home/users/u6642247/anaconda3/envs/ML/include/python3.10 -c dice6_cy3.c -o build/temp.linux-x86_64-cpython-310/dice6_cy3.o -fopenmp
gcc -pthread -B /home/users/u6642247/anaconda3/envs/ML/compiler_compat -shared -Wl,-rpath,/home/users/u6642247/anaconda3/envs/ML/lib -Wl,-rpath-link,/home/users/u6642247/an

In [11]:
from datetime import datetime

N = 500000
ndice = 20
nsix = 5
t0 = datetime.now()
p = dice6_cy3(N, ndice, nsix)
t1 = datetime.now()
print(p, t1-t0)

0.23080399632453918 0:00:02.990716


Surprisingly, it runs slower than our non-parallelised version. 

The issue is that rand() is not thread-safe and hence is implemented into cython with blocking. Provided in `dice6_cy4.pyx` is an alternative way of sampling random numbers using C++.

TODO:
Read through `dice6_cy4.pyx` and `dice6_cy4_setup.py`. Try to answer the following questions for yourself (may be worth Googling some of these):

- How was the Extension changed in `dice6_cy4_setup.py` to now compile C++?
- What do the extra compile and link arguments do?
- what is the C++ library that we import?
- What does the mt19937 class do?
- How does the uniform_real_distribution class work?

If you are unfamiliar with C++, don't worry about understanding how the external C++ classes work.

In [13]:
!python dice6_cy4_setup.py build_ext --inplace
from dice6_cy4 import dice6_cy4

running build_ext
skipping 'dice6_cy4.cpp' Cython extension (up-to-date)


In [14]:
from datetime import datetime


N = 500000
ndice = 100
nsix = 20
t0 = datetime.now()
p = dice6_cy4(N, ndice, nsix)
t1 = datetime.now()
print(p, t1-t0)

0.2189899981021881 0:00:00.244629


Much faster!

### Final Comparisons

Run the below code to see a summary of the performance of the different implementations.

In [ ]:
N = 500000
ndice = 20
nsix = 5

print(f"solution               : {true_prob(ndice, nsix):.6f}")

t0 = datetime.now()
p = dice6_py(N, ndice, nsix)
t1 = datetime.now()
print(f"pure python            : {p:.6f}, {t1-t0}")


t0 = datetime.now()
p = dice6_vec2(N, ndice, nsix)
t1 = datetime.now()
print(f"vectorized python      : {p:.6f}, {t1-t0}")


t0 = datetime.now()
p = dice6_cy1(N, ndice, nsix)
t1 = datetime.now()
print(f"plain cython           : {p:.6f}, {t1-t0}")


t0 = datetime.now()
p = dice6_cy2(N, ndice, nsix)
t1 = datetime.now()
print(f"cython + stdlib rand   : {p:.6f}, {t1-t0}")


t0 = datetime.now()
p = dice6_cy3(N, ndice, nsix)
t1 = datetime.now()
print(f"cython prange + rand   : {p:.6f}, {t1-t0}")


t0 = datetime.now()
p = dice6_cy4(N, ndice, nsix)
t1 = datetime.now()
print(f"cython prange + cpp rng: {p:.6f}, {t1-t0}")

### Wrapping C/C++ with Cython

As an extension task, try and implement the dice problem in C++ simply using the standard library `rand()` function. Then write a wrapper using Cython before compiling and calling it here.